In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load the Breast Cancer Wisconsin dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for training
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the Neural Network (NN) model
class NN(nn.Module):
    def __init__(self, n_input, n_output):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(n_input, 100)
        self.fc2 = nn.Linear(100, n_output)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))

# Train an ensemble of NN models
n_models = 10
models = []
for i in range(n_models):
    model = NN(X_train.shape[1], 1)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    num_epochs = 1000

    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.view(-1, 1))
            loss.backward()
            optimizer.step()
    models.append(model)

# Test the ensemble model
ensemble_preds = []
for model in models:
    model.eval()
    with torch.no_grad():
        preds = model(X_test)
        ensemble_preds.append(preds)
ensemble_preds = torch.cat(ensemble_preds, dim=-1)
average_preds = ensemble_preds.mean(dim=-1)

# Calculate the accuracy
accuracy = (average_preds.round() == y_test).float().mean()
print(f"Ensemble accuracy: {accuracy:.4f}")

Ensemble accuracy: 0.9649
